In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

In [16]:
# Load transaction data from the chosen file
tr = pd.read_excel('../data/bank.xlsx')
tr

,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1.000000e+06,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2.000000e+06,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2.500000e+06,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5.500000e+06,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6.000000e+06,.
...,...,...,...,...,...,...,...,...,...
116196,409000362497',2019-03-05,TRF TO 1196428 Indiaforensic SE,NaN,2019-03-05,117934.30,NaN,-1.901902e+09,.
116197,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901602e+09,.
116198,409000362497',2019-03-05,FDRL/INTERNAL FUND TRANSFE,NaN,2019-03-05,NaN,300000.0,-1.901302e+09,.
116199,409000362497',2019-03-05,IMPS 05-03-20194C,NaN,2019-03-05,109868.65,NaN,-1.901412e+09,.


In [17]:
tr.shape

(116201, 9)

In [18]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116201 entries, 0 to 116200
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Account No           116201 non-null  object        
 1   DATE                 116201 non-null  datetime64[ns]
 2   TRANSACTION DETAILS  113702 non-null  object        
 3   CHQ.NO.              905 non-null     float64       
 4   VALUE DATE           116201 non-null  datetime64[ns]
 5   WITHDRAWAL AMT       53549 non-null   float64       
 6   DEPOSIT AMT          62652 non-null   float64       
 7   BALANCE AMT          116201 non-null  float64       
 8   .                    116201 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(3)
memory usage: 8.0+ MB


In [19]:
tr.isnull().sum()

Account No                  0
DATE                        0
TRANSACTION DETAILS      2499
CHQ.NO.                115296
VALUE DATE                  0
WITHDRAWAL AMT          62652
DEPOSIT AMT             53549
BALANCE AMT                 0
.                           0
dtype: int64

In [20]:



from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
tr["Account No"] = encoder.fit_transform(tr["Account No"])
tr["TRANSACTION DETAILS"] = encoder.fit_transform(tr["TRANSACTION DETAILS"])
tr.drop('CHQ.NO.', axis=1, inplace=True)
tr.drop('.', axis=1, inplace=True)
# Convert date columns to datetime
tr['DATE'] = pd.to_datetime(tr['DATE'], format='%d-%b-%y')
tr['VALUE DATE'] = pd.to_datetime(tr['VALUE DATE'], format='%d-%b-%y')

# Convert datetime columns to a numeric value, e.g., days since a reference date
reference_date = pd.Timestamp('2010-01-01')
tr['DATE_numeric'] = (tr['DATE'] - reference_date).dt.days
tr['VALUE_DATE_numeric'] = (tr['VALUE DATE'] - reference_date).dt.days

# Convert 'WITHDRAWAL AMT', 'DEPOSIT AMT', and 'BALANCE AMT' to numeric, assuming they are separated by commas
tr['WITHDRAWAL AMT'] = pd.to_numeric(tr['WITHDRAWAL AMT'].replace(',', '', regex=False), errors='coerce')
tr['DEPOSIT AMT'] = pd.to_numeric(tr['DEPOSIT AMT'].replace(',', '', regex=False), errors='coerce')
tr['BALANCE AMT'] = pd.to_numeric(tr['BALANCE AMT'].replace(',', '', regex=False), errors='coerce')

TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['float', 'int', 'str']

In [ ]:
# Now include them in the numeric_columns
numeric_columns = ['Account No', 'TRANSACTION DETAILS', ' WITHDRAWAL AMT ', ' DEPOSIT AMT ', 'BALANCE AMT', 'DATE_numeric', 'VALUE_DATE_numeric']
tr_numeric = tr[numeric_columns]

In [ ]:
tr.drop('DATE', axis=1, inplace=True)
tr.drop('VALUE DATE', axis=1, inplace=True)
tr.fillna(0, inplace=True)

In [ ]:
tr.info()

In [ ]:
tr

In [ ]:
# Assuming tr is your DataFrame and the date columns have been converted to numeric as per the previous step
scaler = StandardScaler()
tr_scaled = scaler.fit_transform(tr)

In [ ]:
# DBSCAN clustering
eps_values = [0.1, 0.5, 1.0]  
min_samples_values = [5, 10, 15]  

best_score = -1
best_params = None
best_labels = None

# Number of clusters, ignoring noise
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(tr_scaled)
        score = len(set(labels)) - (1 if -1 in labels else 0)  
        if score > best_score:
            best_score = score
            best_params = (eps, min_samples)
            best_labels = labels

In [ ]:
# Extracting the best parameters and clustering results
best_eps, best_min_samples = best_params
print("Best parameters: eps={}, min_samples={}".format(best_eps, best_min_samples))
print("Number of clusters found:", best_score)

In [ ]:
# Assigning cluster labels to the original data
tr['cluster'] = best_labels

In [ ]:
# Accessing clustered transactions
for cluster_id in tr['cluster'].unique():
    cluster_transactions = tr[tr['cluster'] == cluster_id]
    print(f"Cluster {cluster_id}:")
    print(cluster_transactions.head())

In [ ]:
# Analyzing anomalies
anomaly_mask = best_labels == -1  
anomalies = tr[anomaly_mask]

In [ ]:
# Analyzing characteristics of anomalies
anomalies_description = anomalies.describe()
print("Characteristics of anomalies:")
print(anomalies_description)

In [ ]:
# Visualizing clusters and outliers
plt.figure(figsize=(10, 6))

# Plotting clustered transactions
plt.scatter(tr['BALANCE AMT'], tr['DATE_numeric'], c=labels, cmap='viridis', alpha=0.5)
plt.colorbar(label='Cluster')
plt.title('DBSCAN Clustering of Transactions')
plt.xlabel('BALANCE AMT')
plt.ylabel('DATE_numeric')
plt.grid(True)

# Highlighting anomalies
plt.scatter(anomalies['BALANCE AMT'], anomalies['DATE_numeric'], color='red', label='Anomalies')
plt.legend()

plt.show()